# Table of Contents | ASD DATA 

- [The BREAST-CANCER dataset](#The-BREAST-CANCER-dataset):
    - [Load the dataset](#Load-the-Dataset)
    - [Explore the dataset: Descriptive statistics](#Explore-the-dataset:-Descriptive-statistics)
    - [Explore the dataset: Visualization](#Explore-the-dataset:-Visualization)
    



The entire project has been based on the following study [Investigating the Correspondence of Clinical  Diagnostic Grouping With Underlying Neurobiological and Phenotypic Clusters Using Unsupervised Machine Learning](https://doi.org/10.1016/j.dib.2018.01.080).

The work focuses on two different pathologies in brain disorders: ASD and ADHD

## The ASD dataset

available at [ABIDE I database](https://fcon_1000.projects.nitrc.org/indi/abide/abide_I.html).


This is .......
 
This data set includes 286 intances (201 of one class, 85 of another class).  The instances are described by 9 attributes, some of which are ordinal and some are nominal.
 
Attribute information

| column | values |
| --- | --- |
| Class | no-recurrence-events, recurrence-events |
| age | 10-19, 20-29, 30-39, 40-49, 50-59, 60-69, 70-79, 80-89, 90-99|
| menopause | lt40, ge40, premeno|
| tumor-size | 0-4, 5-9, 10-14, 15-19, 20-24, 25-29, 30-34, 35-39, 40-44, 45-49, 50-54, 55-59|
| inv-nodes | 0-2, 3-5, 6-8, 9-11, 12-14, 15-17, 18-20, 21-23, 24-26, 27-29, 30-32, 33-35, 36-39|
| node-caps | yes, no|
| deg-malig | 1, 2, 3|
| breast | left, right|
| breast-quad | left-up, left-low, right-up, right-low, central|
| irradiat | yes, no|
 
There are 9 Missing Attribute Values (denoted by "?") 


In [ ]:
import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt #for the plots
import seaborn as sns 
import re

In [ ]:
ASD_phenotypic = pd.read_csv(os.path.join('DataSets','Phenotypic Datasets','ASD','ASD_phenotypic.csv'))

In [ ]:
#visualizziamo, non completamente, il nostro dataset
#e otteniamone la dimensione: prima info utile
ASD_phenotypic

In [ ]:
ASD_phenotypic.head() #in order to underline features

In [ ]:
#otteniamo le informazioni relative al dataset
ASD_phenotypic.info()

#  CATEGORICAL EXPLORATION

In [ ]:
# Seleziona tutte le colonne di tipo 'object'
object_columns = ASD_phenotypic.select_dtypes(include=['object']).columns

# Converti le colonne selezionate in tipo 'categorical'
ASD_phenotypic[object_columns] = ASD_phenotypic[object_columns].astype('category')


# Seleziona solo le colonne di tipo 'category'
category_columns = ASD_phenotypic.select_dtypes(include=['category'])

# Ciclo su tutte le colonne di tipo 'category'
for column in category_columns.columns:
    # Stampa il nome della colonna
    print("Feature:", column)
    # Stampa i valori unici della colonna
    for value in category_columns[column].unique():
        print(value)
    print()

Exploding categorical features, we notice the presence of value -9999, commonly used to denote missing data or values out of range, so we are going to consider them as NaN.

In [ ]:
# Sostituisci il valore -9999 con NaN in tutto il DataFrame
ASD_phenotypic.replace([-9999, "-9999"], np.nan, inplace=True)
#ASD_phenotypic

In [ ]:
# Ottieni i nomi delle colonne categoriche come una lista
categorical_column_names = category_columns.columns.tolist()



In [ ]:
category_columns.info()

To understand better how to treat the information gived by this categorical variables we are interested in know which values are stored in this features. We will analyze all of them.


## Handling categorical variables

Thanks to the implemented tolist, we can acced to specific elements.
We prefer create viasual subsections in order to manage these features, but we could implement a 'for logic' in order to guarantee a correct automatic work also in case of modifications on dataset.

For each categorical feature, we want to investigate the amount of informations given. We suppose that for our specific scope we could drop some uninformative features, but we want to proof it. In which way? 
- Evaluating the amount of info considering Nan as Nan
- Changing Missing values with specific feature engineering rules
- Evaluating the amount of info with Nan evalueted

We use Entropy logic.
If both have high level of entropy we can drop the feature.

### SITE_ID

SITE_ID refers to the place where the data from the subject was recluted. 


In [ ]:
# Accesso a una specifica colonna categorica utilizzando la lista di nomi
specific_category_column = ASD_phenotypic[categorical_column_names[0]].value_counts(dropna=False)
specific_category_column


There is data that has been collected from the same center that we decide to unify.

In [ ]:
# We create a function to replace the categories for the indicated cases

def replace_categories(category):
    if "UCLA" in category:
        return "UCLA"
    if "LEUVEN" in category:
        return "LEUVEN"
    if "UM" in category:
        return "UM"
    else:
        return category

# Then we apply the replace function
ASD_phenotypic[categorical_column_names[0]] = ASD_phenotypic[categorical_column_names[0]].apply(replace_categories)

# Now we check the new order
specific_category_column = ASD_phenotypic[categorical_column_names[0]].value_counts(dropna=False)
specific_category_column

### HANDEDNESS_CATEGORY

HANDEDNESS_CATEGORY refers to the handedness of the subject. We don't really know if there is a correlation or not between the Autism Disease and the handnedness of the subject and as it is a caracteristic of the subject itselfs and not of the specific site of analysis as in the previous case, we decide to work with this feature.

In [ ]:
# Accesso a una specifica colonna categorica utilizzando la lista di nomi
specific_category_column = ASD_phenotypic[categorical_column_names[1]].value_counts(dropna=False)
specific_category_column
#hiii

We can see that there are incongruences for the Ambidextreus group, so we will replace them (Mixed and L->R) for Ambi.

In [ ]:
# We decide to replace all the values with "Mixed" or "L->R" with "Ambi"
def replace_categories(category):
    if category in ['Mixed', 'L->R']:
        return 'Ambi'
    else:
        return category

# Apply the custom function to the categorical column
ASD_phenotypic[categorical_column_names[1]] = ASD_phenotypic[categorical_column_names[1]].apply(replace_categories)

# Display the new result
specific_category_column = ASD_phenotypic[categorical_column_names[1]].value_counts(dropna=False)
specific_category_column

This should be applied at the final????

We can see that we have values for R, L and Ambi, Mixed, L->R. The dataset include as a feature also a handness score where right-handed subjects has positive score (max = 100), left-handed subjects negative score (min = -100) and ambidextreus subjects has 0 score. 

To be coherent with that categorization and can properly evaluate if one of those features contain redudant information or that can be merged in some manner, we decide to assign to R values the number "100", to L values the number "-100" and to Ambi, Mixed, L->R the number "0".

### FIQ_TEST_TYPE, VIQ_TEST_TYPE and PIQ_TEST_TYPE

FIQ_TEST_TYPE, VIQ_TEST_TYPE and PIQ_TEST_TYPE refers to the type of test that each center chose to get the information of FIQ_TEST, VIQ_TEST and PIQ_TEST respectively. As we want our clustering algorithm to be as most general as possible, we want to be able to categorize subjects in despise of the test used by the centers to get the data. So we decide to drop this feature as well.

Note that if in a future we will be interested in to analyze if there are differences between the clustering score obtained using the result for each difference test we'll can retrieve the information opportunely.

In [ ]:
for i in range (2,5):
    specific_category_column = ASD_phenotypic[categorical_column_names[i]].value_counts(dropna=False)
    print(specific_category_column)
    print('______________________________________\n')

We make some little changes in the categories in order to homogenize the data. NOT WORKINGGGGG

In [ ]:
# We create a function to replace the categories for the indicated cases

def replace_categories(category):
    if "WISC_III" in category:
        return "WISC_III"
    if "PPVT" in category:
        return "PPVT"
    if category in ["wisc4","WISC4", "WISC_IV_FULL", "WISC_IV_4_SUBTESTS"]:
        return "WISC_IV"
    if "Ravens" in category:
        return "PPVT"
    if "WASI" in category:
        return "WASI"
    else:
        return category

for i in range (2,5):
# Then we apply the replace function
    ASD_phenotypic[categorical_column_names[i]] = ASD_phenotypic[categorical_column_names[i]].apply(replace_categories)

    # Now we check the new order
    specific_category_column = ASD_phenotypic[categorical_column_names[i]].value_counts(dropna=False)
    print(specific_category_column)

### COMORBIDITY

COMORBIDITY indicates if the subject present some othe pathology or disease or particular detail that is important to specify.

In [ ]:
column_name = 'COMORBIDITY'

# Get unique values and their frequencies
unique_values_counts = ASD_phenotypic[column_name].value_counts(dropna=False)

# Display unique values and their frequencies
print("Unique values in column '{}' and their frequencies:".format(column_name))
for value, count in unique_values_counts.items():
    print("{}: {}".format(value, count))

We can see that there is a lot of variability between the commorbities and the combinations of them in the patients. We note as well that there is a large quantity of NaN values. To understand better how the data was collected and how to work with it, we want to understand if the large amount of NaN is due to differences in the protocols between different centers of data collection.

In [ ]:
non_nan_counts = ASD_phenotypic.groupby('SITE_ID')['COMORBIDITY'].count()
print(non_nan_counts)


As we anticipated, it seems that only NYU, KKI and SBL collected data about the commorbities. We can also note that they didn't collected it for all their subjects (total subjects for NIU = 184, KKI = 55, SBL = 30). In this way is difficult to understand how to treat the other subjects, because there isn' a clear tendency to follow. We can ipotetize that the other centers didn't collect data about commorbities, but we can't say with security that the missing values for the NYU, KKI and SBL center mean that the other subjects haven't commorbities. 

Taking into account the existent limitance in the available data, we decided that the less risky option is to treat the NaN values as None, while for the other commorbities we will reduce them to macrogroups. As our other dataset is about ADH, the selected macrogroups are: 
- patients that presents a form of ADHD, catalogated as ADHD
- patients that has other disorders, catalogated as OTHER_MENTAL_DISORDER

In [ ]:

# To make this we create a function that is able to treat each case as we defined
import re

def replace_value(x):
    if pd.isna(x) or x == "None":
        return "None"
    elif re.search(r'\bADHD\b', x, flags=re.IGNORECASE):
        return "ADHD"
    else:
        return "OTHER_MENTAL_DISORDER"

# Assuming your DataFrame is named 'data' and the column with strings is named 'feature'
# Apply the function to the 'feature' column
ASD_phenotypic['COMORBIDITY'] = ASD_phenotypic['COMORBIDITY'].apply(replace_value)

Then we check the new distribution

In [ ]:
column_name = 'COMORBIDITY'

# Get unique values and their frequencies
unique_values_counts = ASD_phenotypic[column_name].value_counts(dropna=False)

# Display unique values and their frequencies
print("Unique values in column '{}' and their frequencies:".format(column_name))
for value, count in unique_values_counts.items():
    print("{}: {}".format(value, count))

### CURRENT_MED_STATUS

This feature indicates if the subject is taking any medication or not. If the subject doesn't take any medication is labeled with a "0" in the other case with a "1".

In [ ]:
column_name = 'CURRENT_MED_STATUS'

# Get unique values and their frequencies
unique_values_counts = ASD_phenotypic[column_name].value_counts(dropna=False)

# Display unique values and their frequencies
print("Unique values in column '{}' and their frequencies:".format(column_name))
for value, count in unique_values_counts.items():
    print("{}: {}".format(value, count))

We can see that the only attribute that is not numeric is the "`", we will catalogate it as a NaN.

In [ ]:
ASD_phenotypic['CURRENT_MED_STATUS'] = ASD_phenotypic['CURRENT_MED_STATUS'].replace('`', float('nan'), regex=True)

unique_values_counts = ASD_phenotypic[column_name].value_counts(dropna=False)

print("Unique values in column '{}' and their frequencies:".format(column_name))
for value, count in unique_values_counts.items():
    print("{}: {}".format(value, count))

### MEDICATION_NAME

This feature indicate which are the principals active ingredients of the medication that the patient is taking

In [ ]:
column_name = 'MEDICATION_NAME'

# Get unique values and their frequencies
unique_values_counts = ASD_phenotypic[column_name].value_counts(dropna=False)

# Display unique values and their frequencies
print("Unique values in column '{}' and their frequencies:".format(column_name))
for value, count in unique_values_counts.items():
    print("{}: {}".format(value, count))

We decided to make a categorization of the pharmacs based on the mechanism of action in the following way:

- Antidepressant: Citalopram (Citaopram)(Citolopram), Fluoxetine, Sertraline, Escitalopram, Trazodone, Peroxatine (Paroxetine), Antidepressant,Venlafaxine, Mirtazapine, Venlafaxine, Duloxetine, Buspirone

- Neurostimulant (ADHD): Methylphenidate (methlphenidate)(Methylphanidate)(Merthylphenidate)(Methylphenidated)(Metadate), Lisdexamfetamine, Dexmethylphenidate, Dextroamphetamine(Dextramphetamine)(Dexedrine)(andDdextroamphetamine) and amphetamine, Bupropion (Buproprion), Concerta, Atomoxetine, Strattera

- Antipsychotic: Risperidone (Risperdone), Paliperidone, Ziprasidone, Aripiprazole, Asenapine, Quetiapine, Benperidol, Guanfacine, tenex, Clonidine

- Mood stabiliser: Lamotrigine, Oxcarbazepine, Topiramato, Valproic Acid, Altrex, Eszopiclone, Lorazepam, Lithium Carbonate (Lithium), Zolpidem

- Hypothyroidism treatment: Levothyroxine, Synthroid

- Antihypertensives: Lisinopril, Clonidine

- Gastrointestinal medication: Pantoprazole

- Antihistamine: Allegra

- Dietary supplements: Zinc, CoQ10, Melatonin, hydrochloride (HCl)


In [ ]:
# Define dictionaries for each category of medication
categories = {
    "Antidepressant": ["Citalopram", "Citaopram", "Citolopram", "Fluoxetine", "Sertraline", "Escitalopram", "Trazodone", "Peroxatine", "Paroxetine", "Venlafaxine", "Mirtazapine", "Duloxetine", "Buspirone"],
    "Neurostimulant (ADHD)": ["Methylphenidate", "methlphenidate", "Methylphanidate", "Merthylphenidate", "Methylphenidated", "Metadate", "Lisdexamfetamine", "Dexmethylphenidate", "Dextroamphetamine", "Dextramphetamine", "Dexedrine", "andDdextroamphetamine", "amphetamine", "Bupropion", "Buproprion", "Concerta", "Atomoxetine", "Strattera"],
    "Antipsychotic": ["Risperidone", "Risperdone", "Paliperidone", "Ziprasidone", "Aripiprazole", "Asenapine", "Quetiapine", "Benperidol", "Guanfacine", "tenex", "Clonidine"],
    "Mood stabiliser": ["Lamotrigine", "Oxcarbazepine", "Topiramato", "Valproic Acid", "Altrex", "Eszopiclone", "Lorazepam", "Lithium Carbonate", "Lithium", "Zolpidem"],
    "Hypothyroidism treatment": ["Levothyroxine", "Synthroid"],
    "Antihypertensives": ["Lisinopril", "Clonidine"],
    "Gastrointestinal medication": ["Pantoprazole"],
    "Antihistamine": ["Allegra"],
    "Dietary supplements": ["Zinc", "CoQ10", "Melatonin", "hydrochloride", "HCl"]
}

# Create an empty dictionary to store medication counts for each subject
medication_counts = {}

# Iterate over each row in the DataFrame
for index, row in ASD_phenotypic.iterrows():
    subject_id = row['SUB_ID']
    medication_name = row['MEDICATION_NAME']

    # Skip NaN values
    if pd.isnull(medication_name):
        continue
    
    # Count the occurrences of each medication category for the current subject
    category_count = {category: 0 for category in categories}
    for category, meds in categories.items():
        for med in meds:
            if med.lower() in medication_name.lower():
                category_count[category] += 1
    
    # Store the medication counts for the current subject
    medication_counts[subject_id] = category_count

# Convert the dictionary to a DataFrame for visualization
medication_counts_df = pd.DataFrame.from_dict(medication_counts)

# Display the DataFrame
print(medication_counts_df)


In [ ]:
# Plotting
plt.figure(figsize=(12, 8))
medication_counts_df.sum(axis=1).sort_values(ascending=False).plot(kind='bar', color='skyblue')
plt.title('Total Medication Counts for Each Subject')
plt.xlabel('Subject ID')
plt.ylabel('Total Medication Count')
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

### Explore the dataset: Descriptive statistics

In [ ]:
ASD_phenotypic.describe()

# NUMERICAL EXPLORATION

The dataset has 1112 raws anf 74 colums

In [ ]:
ASD_phenotypic.shape

Now we check the presence of missing values catalogated as None or numpy.NaN

In [ ]:
nan_values = ASD_phenotypic.isna().sum()

# Filter columns with NaN values
columns_with_nan = nan_values[nan_values > 0]


# Print the number of attributes with NaN values
print("Number of attributes with NaN values:", len(columns_with_nan))

# Print the columns with NaN values and their corresponding counts
print("Attributes with NaN values and their counts:")
pd.set_option('display.max_rows', 74)
columns_with_nan



Distribution of NANs

In [ ]:
nan_values.plot(kind='barh', figsize=(12,12), title='Missing values per feature')
plt.xlabel('Number of missing values')
plt.ylabel('Features')
plt.show()

Is there some attribute with only missing values?

In [ ]:

columns_only_nan = nan_values[nan_values == ASD_phenotypic.shape[0]]
print(len(columns_only_nan))

As there are too much, maybe is usefull to understand which columns haven't NaN values.

In [ ]:
# Filter columns without NaN values
columns_without_nan = nan_values[nan_values == 0]

# Print the columns with NaN values and their corresponding counts
print("Attributes without NaN values and their counts:")
columns_without_nan

There are maybe some subjects that has only missing values?

In [ ]:
nan_values = ASD_phenotypic.T.isna().sum()

# Filter columns with NaN values
subjects_with_nan = nan_values[nan_values > 0]

print("Max missing values encountered for a subject: " +str(max(subjects_with_nan)))
print("Min missing values encountered for a subject: " +str(min(nan_values)))
subjects_with_nan.plot(kind='hist', bins=22, figsize=(10,6), title='Missing values per subject')
plt.show()


Let's have a look on the general statistics for the numerical attributes.

In [ ]:
ASD_phenotypic.describe()

We need to handle missing values. But how?
Is really necessary to fullfill all of them? Can we maybe make some feature selection previously?


To make random forest for features selections

In [ ]:
'''# Assuming 'data' is your DataFrame and 'target' is your target variable
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data.drop(columns=['target']), data['target'], test_size=0.2, random_state=42)

# Convert categorical variables to numeric using one-hot encoding (if needed)
# For example:
# X_train = pd.get_dummies(X_train)

# Train the Random Forest model
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)

# Get feature importance scores
feature_importances = rf_model.feature_importances_

# Optionally, visualize feature importances
# (e.g., create a bar plot with feature names on the x-axis and importance scores on the y-axis)'''

In [ ]:
#vogliamo individuare e capire cosa sono questi object
numeric = ASD_phenotypic.select_dtypes(include=['float64',"int64"]).dropna()
numeric.T
f,ax = plt.subplots(figsize=(10, 8))
sns.heatmap(numeric.drop('DX_GROUP',axis = 1).corr(), 
            annot=True, 
            linewidths=.5, 
            fmt= '.2f',
            ax=ax,
            vmin=-1, 
            vmax=1,
            cmap = "coolwarm")
plt.show()

Drop attribute if:
- Only unique values
- Only missing values

# Entropy

In [ ]:
'''
# Calcola la distribuzione delle frequenze dei valori non mancanti nella feature 'SITE_ID'
site_id_counts = ASD_phenotypic[categorical_column_names[0]].value_counts(dropna=True)

# Calcola la proporzione di ciascun valore rispetto al totale dei dati
site_id_proportions = site_id_counts / site_id_counts.sum()

# Calcola l'entropia della distribuzione dei valori non mancanti
site_id_entropy = -(site_id_proportions * np.log2(site_id_proportions)).sum()

# Stampa l'entropia della feature 'SITE_ID'
print('Considering the feature')
print("Entropia di SITE_ID:", site_id_entropy)
'''

## Entropy Function

In [ ]:
def calculate_entropy(column):
    # Sostituisci i valori NaN con "Missing" solo se la colonna è di tipo object
    if column.dtype == 'category':
        # Aggiungi "Missing" alle categorie esistenti
        column = column.cat.add_categories("Missing")
        column_filled = column.fillna("Missing")
    else:
        column_filled = column.fillna("Missing")  # Se è numerica
        
    # Calcola la distribuzione di probabilità delle categorie
    value_counts = column_filled.value_counts()
    probabilities = value_counts / value_counts.sum()
    
    # Calcola l'entropia
    entropy = -np.sum(probabilities * np.log2(probabilities))
    
    return entropy


## Entropy with "Missing"

In [ ]:
# Calcola l'entropia per tutte le colonne
all_entropy = {}
for column in ASD_phenotypic.columns:
    all_entropy[column] = calculate_entropy(ASD_phenotypic[column])

# Ordina il dizionario in base ai valori di entropia in ordine decrescente
sorted_entropy = {k: v for k, v in sorted(all_entropy.items(), key=lambda item: item[1], reverse=True)}

# Plot dell'entropia per tutte le colonne
# Definisci i colori per le feature categoriche e numeriche
color_categorical = 'skyblue'
color_numerical = 'lightgreen'
plt.figure(figsize=(12, 6))
plt.bar(sorted_entropy.keys(), sorted_entropy.values(), color=[color_categorical if col in category_columns.columns else color_numerical for col in sorted_entropy.keys()])
plt.xticks(rotation=90)
plt.xlabel('Feature')
plt.ylabel('Entropy')
plt.title('Entropy of All Features (NaN treated as "Missing" for object types)')
plt.show()

## Entropy with NaN 

In [ ]:
# Calcolo dell'entropia per tutte le colonne
all_entropy_dropna = {}

for column in ASD_phenotypic.columns:
    # Rimuovi i valori NaN dalla colonna
    column_without_nan = ASD_phenotypic[column].dropna()
    
    # Se la colonna è vuota dopo aver rimosso i NaN, imposta l'entropia a 0
    if column_without_nan.empty:
        entropy = 0
    else:
        # Calcola la distribuzione di probabilità delle categorie
        value_counts = column_without_nan.value_counts()
        probabilities = value_counts / value_counts.sum()
        
        # Calcola l'entropia
        entropy = -np.sum(probabilities * np.log2(probabilities))
    # Memorizza l'entropia calcolata per la colonna
    all_entropy_dropna[column] = entropy

# Plot dell'entropia per tutte le colonne
# Ordina il dizionario in base ai valori di entropia in ordine decrescente
sorted_entropy_dropna = {k: v for k, v in sorted(all_entropy_dropna.items(), key=lambda item: item[1], reverse=True)}
plt.figure(figsize=(12, 6))
plt.bar(sorted_entropy_dropna.keys(), sorted_entropy_dropna.values(), color=[color_categorical if col in category_columns.columns else color_numerical for col in sorted_entropy.keys()])
plt.xticks(rotation=90)
plt.xlabel('Feature')
plt.ylabel('Entropy')
plt.title('Entropy of All Features (NaNs Ignored)')
plt.show()


## Comparison between two entropies

In [ ]:
# Sovrapposizione dei due plot

# Calcola le differenze tra le entropie con e senza NaN per ogni feature
differences = {}
for column in all_entropy.keys():
    difference = all_entropy_dropna[column] - all_entropy[column]
    differences[column] = difference

# Crea il grafico a dispersione delle differenze
plt.figure(figsize=(12, 6))
plt.scatter(range(len(differences)), list(differences.values()), color='skyblue')
plt.axhline(y=0, color='red', linestyle='--', linewidth=1)  # Aggiungi una linea orizzontale a y=0 per la visualizzazione
plt.xlabel('Feature')
plt.ylabel('Difference in Entropy (Drop NaN - Keep NaN)')
plt.title('Scatter Plot of Entropy Differences (Drop NaN vs Keep NaN)')
plt.xticks(range(len(differences)), list(differences.keys()), rotation=90)
plt.grid(True, axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()
